In [ ]:
# 1.	Import pandas, numpy 

In [1]:
#Load packages
import pandas as pd
import numpy as np

# I. Comparing Event Run Values

In [2]:
# Read in MLBAM Data for 2014-2017

MLBAM14 = pd.read_csv("../Data/MLBAM14.csv")
MLBAM15 = pd.read_csv("../Data/MLBAM15.csv")
MLBAM16 = pd.read_csv("../Data/MLBAM16.csv")
MLBAM17 = pd.read_csv("../Data/MLBAM17.csv")

In [5]:
MLBAM14

,Unnamed: 0,inning,batterId,pitcherId,event,x,y,ab_num,timestamp,stand,...,fielderId,gameId,isPA,isAB,isHit,isBIP,our.x,our.y,r,theta
0,1,1,624577,489119,Strikeout,NaN,NaN,1,2014-03-22 09:00:25,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,False,NaN,NaN,NaN,NaN
1,2,1,457759,489119,Strikeout,NaN,NaN,2,2014-03-22 09:01:45,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,False,NaN,NaN,NaN,NaN
2,3,1,434670,489119,Groundout,139.56,160.64,3,2014-03-22 09:03:23,R,...,431094.0,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,True,36.336970,95.733939,102.398059,1.208032
3,4,1,572041,477132,Strikeout,NaN,NaN,4,2014-03-22 09:08:01,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,False,NaN,NaN,NaN,NaN
4,5,1,431094,477132,Single,110.44,152.61,5,2014-03-22 09:09:10,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,True,True,-36.336970,115.774177,121.342637,1.874920
5,6,1,502671,477132,Single,79.32,113.45,6,2014-03-22 09:10:36,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,True,True,-114.002251,213.504653,242.034605,2.061239
6,7,1,445988,477132,Groundout,115.46,187.75,7,2014-03-22 09:11:58,R,...,477132.0,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,True,-23.808701,28.076299,36.812128,2.274128
7,8,1,444432,477132,Groundout,109.44,158.63,8,2014-03-22 09:13:38,R,...,434670.0,gid_2014_03_22_lanmlb_arimlb_1,True,True,False,True,-38.832641,100.750238,107.974925,1.938684
8,9,2,408236,489119,Walk,NaN,NaN,9,2014-03-22 09:17:44,L,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,False,False,False,NaN,NaN,NaN,NaN
9,10,2,475100,489119,Double,59.24,62.25,10,2014-03-22 09:19:14,R,...,NaN,gid_2014_03_22_lanmlb_arimlb_1,True,True,True,True,-164.115324,341.283008,378.692397,2.019029


In [ ]:
#Your Code Here

In [ ]:
# 2.	Using a function similar to what was used in the week’s exercise notebook, 
# calculate the run value for every event in 2014, 2015, 2016 and 2017

In [3]:
def Run_Expectancy(path):
    
    RE = pd.read_csv(path)
    RE.drop(['Unnamed: 0'], axis=1, inplace=True)
    RE = RE[['home_team','away_team','half','gameId','batterName','batterId','event', 'start1B', 'start2B', 'start3B',\
             'end1B', 'end2B', 'end3B', 'startOuts','endOuts','runsFuture','runsOnPlay','outsInInning','venueId','batterPos']]
    RE['Start1'] = np.where(pd.isnull(RE['start1B']),0,1)
    RE['Start2'] = np.where(pd.isnull(RE['start2B']),0,1)
    RE['Start3'] = np.where(pd.isnull(RE['start3B']),0,1)
    RE['Start_State'] = (RE['Start1'].astype(str) + RE['Start2'].astype(str) + RE['Start3'].astype(str)+\
                          " " + RE['startOuts'].astype(str))
    RE['End1'] = np.where(pd.isnull(RE['end1B']),0,1)
    RE['End2'] = np.where(pd.isnull(RE['end2B']),0,1)
    RE['End3'] = np.where(pd.isnull(RE['end3B']),0,1)
    RE['End_State'] = (RE['End1'].astype(str) + RE['End2'].astype(str) + RE['End3'].astype(str) + \
                        " " + RE['endOuts'].astype(str))
    RE = RE[((RE.Start_State != RE.End_State) | (RE.runsOnPlay > 0)) & (RE.outsInInning == 3)]
    Start_RunExp = RE.groupby(['Start_State'])['runsFuture'].mean().reset_index().rename(columns={'runsFuture':'Start_RE'})
    RE = pd.merge(RE, Start_RunExp, on=['Start_State'], how='left')
    Base_State_3 = [pd.Series(['000 3', 0], index=Start_RunExp.columns),
                pd.Series(['001 3', 0], index=Start_RunExp.columns),
                pd.Series(['010 3', 0], index=Start_RunExp.columns),
                pd.Series(['011 3', 0], index=Start_RunExp.columns),
                pd.Series(['100 3', 0], index=Start_RunExp.columns),
                pd.Series(['101 3', 0], index=Start_RunExp.columns),
                pd.Series(['110 3', 0], index=Start_RunExp.columns),
                pd.Series(['111 3', 0], index=Start_RunExp.columns)]
    Start_RunExp = Start_RunExp.append(Base_State_3, ignore_index=True)
    End_RunExp  = Start_RunExp.rename(columns={'Start_State':'End_State', 'Start_RE':'End_RE'})
    RE = pd.merge(RE, End_RunExp, on=['End_State'], how='left')
    RE['Run_Value'] = RE['runsOnPlay'] + RE['End_RE'] - RE['Start_RE']
    
    return RE;

In [ ]:
# 3.	Calculate the average run value for each type of event for every season

In [7]:
# Calculate run value for every event in seasons 2014-2017
RE_14 = Run_Expectancy("../Data/MLBAM14.csv") 
RE_15 = Run_Expectancy("../Data/MLBAM15.csv")                          
RE_16 = Run_Expectancy("../Data/MLBAM16.csv") 
RE_17 = Run_Expectancy("../Data/MLBAM17.csv") 

In [10]:
# Average run value for each type of event for every season
Event_Value14 = RE_14.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV14'})
Event_Value15 = RE_15.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV15'})
Event_Value16 = RE_16.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV16'})
Event_Value17 = RE_17.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV17'})

In [ ]:
# 4.	Merge the event level run values for each season into one data frame. 
# The data frame should include the event name and then four columns with the run values (one for each season)

In [11]:
# Merge event run values into one dataframe
Event_Value = pd.merge(Event_Value14, Event_Value15, on=['event'], how='left')
Event_Value = pd.merge(Event_Value, Event_Value16, on=['event'], how='left')
Event_Value = pd.merge(Event_Value, Event_Value17, on=['event'], how='left')
display(Event_Value)

,event,RV14,RV15,RV16,RV17
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070
5,Double,0.737518,0.752039,0.743467,0.779338
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164
7,Fan interference,0.633560,0.577453,0.533316,0.590743
8,Field Error,0.462976,0.485460,0.469989,0.493206
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112


In [ ]:
# 5.	Delete the row for event “Sacrifice Bunt DP”

In [12]:
# Drop row for Sacrifice Bunt DP
Event_Value = Event_Value[Event_Value.event != "Sacrifice Bunt DP"].copy()
display(Event_Value)

,event,RV14,RV15,RV16,RV17
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070
5,Double,0.737518,0.752039,0.743467,0.779338
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164
7,Fan interference,0.633560,0.577453,0.533316,0.590743
8,Field Error,0.462976,0.485460,0.469989,0.493206
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112


In [ ]:
# 6.	Compute the correlation matrix for event level run values across all seasons

In [13]:
# Compute correlation matrix
Event_Value.corr()

,RV14,RV15,RV16,RV17
RV14,1.000000,0.998819,0.997951,0.997153
RV15,0.998819,1.000000,0.997570,0.996412
RV16,0.997951,0.997570,1.000000,0.994964
RV17,0.997153,0.996412,0.994964,1.000000


In [ ]:
# Q1: Which two seasons have the strongest correlation between run values?

In [ ]:
# 7.	For each event, calculate the sum of squares between run values using the data from all four seasons and create a column for this sum of squares variable
# Run Value Sum of Squares = (RV14 – RVm)2 + (RV15 – RVm)2 + (RV16 – RVm)2 + (RV17 – RVm)2, where RVm = mean run value of event across all seasons

In [14]:
# Calculate sum of squares
Event_Value['Avg_RV'] = (Event_Value['RV14'] + Event_Value['RV15'] + Event_Value['RV16'] + Event_Value['RV17'])/4
Event_Value['SumSq'] = (Event_Value['RV14']-Event_Value['Avg_RV'])**2 + (Event_Value['RV15']-Event_Value['Avg_RV'])**2 + (Event_Value['RV16']-Event_Value['Avg_RV'])**2 + (Event_Value['RV17']-Event_Value['Avg_RV'])**2
display(Event_Value)

,event,RV14,RV15,RV16,RV17,Avg_RV,SumSq
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019,-0.349533,0.011787
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411,-0.205842,0.000334
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292,-0.351493,0.007724
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225,-0.346713,0.001694
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070,0.349826,0.006675
5,Double,0.737518,0.752039,0.743467,0.779338,0.753091,0.001025
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164,-0.861396,0.002402
7,Fan interference,0.633560,0.577453,0.533316,0.590743,0.583768,0.005113
8,Field Error,0.462976,0.485460,0.469989,0.493206,0.477908,0.000577
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112,0.720747,0.002769


In [15]:
# Q2: Which event has the highest sum of squares value?
max(Event_Value['SumSq'])

0.025962645475026698

In [ ]:
# Q3: What was the average run value of a “Flyout” in 2014?

# II. Comparing Player Run Values

In [ ]:
#Your Code Here

In [ ]:
# 1.	Compute the aggregate player level run values for each season

In [23]:
# Aggregate player level run values for each season
Player_Value14 = RE_14.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV14'})
Player_Value15 = RE_15.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV15'})
Player_Value16 = RE_16.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV16'})
Player_Value17 = RE_17.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV17'})

In [ ]:
# 2.	Merge player run values for each season into one data frame so that only players with run values for all four seasons are included in the data frame. 
# The data frame should include each player’s name and then four columns with the run values (one for each season)

In [24]:
# Merge player run values into one data frame
Player_Value = pd.merge(Player_Value14, Player_Value15, on=['batterId','batterName'])
Player_Value = pd.merge(Player_Value, Player_Value16, on=['batterId','batterName'])
Player_Value = pd.merge(Player_Value, Player_Value17, on=['batterId','batterName'])
display(Player_Value)

,batterId,batterName,RV14,RV15,RV16,RV17
0,112526,Colon,-0.709186,-8.838568,-15.380841,-6.053060
1,134181,Beltre,29.004125,11.918638,29.745115,27.205050
2,136860,Beltran,-6.208773,2.355753,21.279393,-16.883042
3,150029,Werth,37.882267,-3.377653,6.368448,-7.246145
4,282332,Sabathia,-1.118087,-0.472414,-0.944580,-0.292495
5,285079,Dickey,-2.106986,-0.858198,-0.291686,-11.955467
6,400085,"Suzuki, I",-3.235189,-24.883224,-2.577561,1.083044
7,400121,"Martinez, V",54.087837,-10.998931,7.335464,-16.021087
8,400284,Utley,15.964367,-12.196781,6.689794,-1.799883
9,405395,Pujols,17.629365,9.840138,15.581010,-13.892055


In [ ]:
# 3.	Compute the correlation matrix for event level run values across all seasons

In [25]:
# Compute correlation matrix
Player_Value.corr()

,batterId,RV14,RV15,RV16,RV17
batterId,1.000000,-0.144210,-0.003344,-0.056668,0.091117
RV14,-0.144210,1.000000,0.466299,0.426629,0.322764
RV15,-0.003344,0.466299,1.000000,0.546136,0.510132
RV16,-0.056668,0.426629,0.546136,1.000000,0.457391
RV17,0.091117,0.322764,0.510132,0.457391,1.000000


In [ ]:
# Q1: What was the correlation in player run values between 2014 and 2016?

In [ ]:
# 4.	Run a regression model by regressing player run values from 2017 (dependent variable) on player run values from 2014, 2015, and 2016 (independent variables)

In [26]:
# Regression model
import statsmodels.formula.api as smf

RV_Reg = smf.ols(formula = 'RV17 ~ RV14 + RV15 + RV16', data=Player_Value).fit()
RV_Reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   RV17   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     54.61
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           3.20e-29
Time:                        19:46:23   Log-Likelihood:                -1458.4
No. Observations:                 372   AIC:                             2925.
Df Residuals:                     368   BIC:                             2941.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0472      0.650      0.073      0.942      -1.231       1.326
RV14           0.0620      0.056      1.101      0.272      -0.049       0.173
RV15           0.3509      0.054      6.480      0.000       0.244       0.457
RV16           0.2673      0.059      4.532      0.000       0.151       0.383
==============================================================================
Omnibus:                       39.399   Durbin-Watson:                   1.924
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.833
Skew:                           0.548   Prob(JB):                     3.11e-20
Kurtosis:                       5.144   Cond. No.                         20.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Q2: What is the R-squared for the regression model run in step 4?

In [ ]:
# Q3: What is the regression coefficient of RV15 when used as an independent variable in the regression?

# III. Comparing Team Run Values

In [ ]:
#Your Code Here

In [ ]:
# 1.	For each season’s run expectancy data frame, create a variable “team” to denote the batting team

In [27]:
# Create team variable in each data frame
RE_14['team']= np.where(RE_14['half']=='top',RE_14['away_team'],RE_14['home_team'])
RE_15['team']= np.where(RE_15['half']=='top',RE_15['away_team'],RE_15['home_team'])
RE_16['team']= np.where(RE_16['half']=='top',RE_16['away_team'],RE_16['home_team'])
RE_17['team']= np.where(RE_17['half']=='top',RE_17['away_team'],RE_17['home_team'])

In [ ]:
# 2.	Compute the aggregate team level run values for each season

In [28]:
# Compute aggregate team level run values by season
REteam_14= RE_14.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV14'})
REteam_15= RE_15.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV15'})
REteam_16= RE_16.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV16'})
REteam_17= RE_17.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV17'})

In [ ]:
# 3.	Merge team run values for each season into one data frame. 
# The data frame should include each team’s name and then four columns with the run values (one for each season)

In [29]:
# Merge run values into one dataframe
REteam = pd.merge(REteam_14, REteam_15, on=['team'])
REteam = pd.merge(REteam, REteam_16, on=['team'])
REteam = pd.merge(REteam, REteam_17, on=['team'])
display(REteam)

,team,RV14,RV15,RV16,RV17
0,ana,104.641532,-25.056067,-3.184209,-47.579782
1,ari,-48.253012,31.924551,15.870968,62.616716
2,atl,-81.885752,-108.464134,-84.633901,-28.259905
3,bal,46.586965,29.882644,29.297940,-23.678031
4,bos,-37.383914,65.372953,160.812546,9.445347
5,cha,-1.307578,-79.136738,-41.643638,-47.416033
6,chn,-54.987532,-3.627047,85.327151,71.002092
7,cin,-57.227567,-61.401274,-17.643638,5.420218
8,cle,-3.725729,-4.178644,48.815791,73.780465
9,col,92.456063,47.127799,122.825528,78.616716


In [ ]:
# 4.	Compute the correlation matrix for event level run values across all seasons

In [30]:
# Compute correlation matrix
REteam.corr()

,RV14,RV15,RV16,RV17
RV14,1.000000,0.363681,0.261814,0.065225
RV15,0.363681,1.000000,0.437610,0.193061
RV16,0.261814,0.437610,1.000000,0.351708
RV17,0.065225,0.193061,0.351708,1.000000


In [ ]:
# Q1: What was the correlation in team run values between 2014 and 2017?

In [ ]:
# 5.	Run a regression model by regressing team run values from 2017 (dependent variable) on team run values from 2014, 2015, and 2016 (independent variables)

In [31]:
# Regression model
TeamRV_Reg = smf.ols(formula = 'RV17 ~ RV14 + RV15 + RV16', data=REteam).fit()
TeamRV_Reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   RV17   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     1.265
Date:                Thu, 19 Dec 2024   Prob (F-statistic):              0.307
Time:                        20:08:51   Log-Likelihood:                -166.93
No. Observations:                  30   AIC:                             341.9
Df Residuals:                      26   BIC:                             347.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4437     12.406     -0.036      0.972     -25.944      25.056
RV14          -0.0553      0.241     -0.230      0.820      -0.550       0.440
RV15           0.0706      0.240      0.294      0.771      -0.424       0.565
RV16           0.3788      0.231      1.638      0.113      -0.097       0.854
==============================================================================
Omnibus:                        0.584   Durbin-Watson:                   1.712
Prob(Omnibus):                  0.747   Jarque-Bera (JB):                0.061
Skew:                           0.065   Prob(JB):                        0.970
Kurtosis:                       3.179   Cond. No.                         77.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Q2: What is the R-squared for the regression model run in step 5?

In [ ]:
# Q3: What is the regression coefficient of RV16 when used as an independent variable in the regression??

In [ ]:
# Q4: Which independent variable(s) had coefficients that were significant in the player level regression but insignificant in the team level regression (at the .05 significance level)?